In [1]:
import tensorflow as tf
import pprint
import pycurl
from io import BytesIO
import json
import time
import matplotlib.pyplot as plt
import numpy as np

/home/hudz0r/miniconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Load model from file

In [2]:
sess = tf.Session()
# we want the variable values from from my_tf_model (not working right now)
saver = tf.train.import_meta_graph('models/my_tf_model.meta')
saver.restore(sess, tf.train.latest_checkpoint('models'))

graph = tf.get_default_graph()

# init variables from trained model

print(len(graph.get_collection('trainable_variables')))

W_conv1 = graph.get_collection('trainable_variables')[10]
b_conv1 = graph.get_collection('trainable_variables')[11]
W_conv2 = graph.get_collection('trainable_variables')[12]
b_conv2 = graph.get_collection('trainable_variables')[13]
W_conv3 = graph.get_collection('trainable_variables')[14]
b_conv3 = graph.get_collection('trainable_variables')[15]
W_dense1 = graph.get_collection('trainable_variables')[16]
b_dense1 = graph.get_collection('trainable_variables')[17]
W_out = graph.get_collection('trainable_variables')[18]
b_out = graph.get_collection('trainable_variables')[19]

# build graph

X = tf.placeholder(tf.float32, [1,112, 200, 3])

pkeep = tf.placeholder(tf.float32)

def build_conv_net(X_in, keep_prob, dropout=0.1610, kernel_size=3, filters=32, hiddenunits=500):
        
    
    conv1 = tf.nn.relu(tf.nn.conv2d(X_in, W_conv1, strides=[1,1,1,1], padding="VALID") + b_conv1)
     
    maxpool1 = tf.layers.max_pooling2d(inputs=conv1,
                                       pool_size=2,
                                       strides=2)
            
    conv2 = tf.nn.relu(tf.nn.conv2d(maxpool1, W_conv2, strides=[1,1,1,1], padding="VALID") + b_conv2)
        
    maxpool2 = tf.layers.max_pooling2d(inputs=conv2,
                                       pool_size=2,
                                       strides=2)
        
    conv3 = tf.nn.relu(tf.nn.conv2d(maxpool2, W_conv3, strides=[1,1,1,1], padding="VALID") + b_conv3)
        
    maxpool3 = tf.layers.max_pooling2d(inputs=conv3,
                                       pool_size=2,
                                       strides=2)
                
    flat = tf.contrib.layers.flatten(inputs=maxpool3)
                
    dropout1 = tf.nn.dropout(flat, pkeep)
        
    dense1 = tf.nn.relu(tf.matmul(dropout1, W_dense1) + b_dense1)
        
    dropout2 = tf.nn.dropout(dense1, pkeep)
        
    out = tf.nn.relu(tf.matmul(dropout2, W_out) + b_out)
    
    return out


INFO:tensorflow:Restoring parameters from models/my_tf_model
20


### Graph info

In [3]:
print(graph.get_all_collection_keys())
print(graph.get_collection('trainable_variables')[8])

['queue_runners', 'summaries', 'trainable_variables', 'train_op', 'variables']
<tf.Variable 'dense_1/kernel:0' shape=(500, 1) dtype=float32_ref>


### Functions

In [4]:
# returns image from webcam with id
def get_img(webcam_id):
    image = plt.imread("data/test_image.jpg")
    image = np.expand_dims(image, axis=0)
    return image

### Main program

In [ ]:
webcam_id_list = [33] #dummy values
sleep_time = 60 #check every minute
net = build_conv_net(X, pkeep)
while(1):
    for w in webcam_id_list:
        i = get_img(w)
        
        count = sess.run(net, feed_dict={X:i, pkeep:1.0})
        print(type(count))
        print("Webcam " + str(w) + " has " + str(count[0].flatten()) + " number of cars")
    time.sleep(sleep_time)

<class 'numpy.ndarray'>
Webcam 33 has [ 42.25443649] number of cars
